####Orquestração
-Utilizado o databricks workflow para execução dos notebooks (Bronze, Siver, Gold)  
-Schedule: diariamente às 06 horas  
-Notificação por email em caso de falha do pipeline  
-Retries: 1x cada task delay 5 min

![](./utils/Brewery_Pipe.png)

##### YAML do Pipeline

In [0]:
"""
resources:
  jobs:
    Brewery_Pipeline:
      name: Brewery_Pipeline
      description: |-
        Pipeline de ingestão dos dados de origem da API openbrewerydb.org.
        Ingestão nas camadas Bronze, Silver, Gold
      email_notifications:
        on_failure:
          - raphael.2205@gmail.com
      schedule:
        quartz_cron_expression: 43 0 6 * * ?
        timezone_id: America/Sao_Paulo
        pause_status: UNPAUSED
      
      tasks:
        - task_key: Bronze
          notebook_task:
            notebook_path: /Workspace/Users/raphael.2205@gmail.com/bees/1-brewery_bronze
            source: WORKSPACE
          max_retries: 1
          min_retry_interval_millis: 300000
          disable_auto_optimization: false
          notification_settings:
            alert_on_last_attempt: true
        
        - task_key: Silver
          depends_on:
            - task_key: Bronze
          notebook_task:
            notebook_path: /Workspace/Users/raphael.2205@gmail.com/bees/2-brewery_silver
            source: WORKSPACE
          max_retries: 1
          min_retry_interval_millis: 300000
          disable_auto_optimization: false
        
        - task_key: Gold
          depends_on:
            - task_key: Silver
          notebook_task:
            notebook_path: /Workspace/Users/raphael.2205@gmail.com/bees/3-brewery_gold
            source: WORKSPACE
          max_retries: 1
          min_retry_interval_millis: 300000
          disable_auto_optimization: false
      
      queue:
        enabled: true
      performance_target: STANDARD
"""

#### Alerta Data Quality

-Criado o alerta **Brewery_silver_check_quality** utilizado o recurso de ALERTA do databricks  
-Verificação diária às 06:30  
-Notificação por E-mail em caso de acionamento do alerta

Regra:  
-Verifica através da view vw_brewery_silver_quality se há regras com 100% dos registros nulos   
e/ou   
-Se a regra is_location_missing possui ocorrencias > 0

![](./utils/Brewery_Alert.png)
